In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

Using TensorFlow backend.


TensorFlow version:  2.1.0


In [2]:
import tensorboard
tensorboard.__version__

'2.1.1'

In [3]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [4]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [5]:
def construct_network(dropout, neuronPct, neuronShrink):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten())
    model.add(Dense(neuronCount))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(14, activation='sigmoid')) # Output
    return model

In [6]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train", class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev", class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)

** compute class weights from training data **
** class_weights **
[{0: 0.976060692178489, 1: 0.023939307821511}, {0: 0.9379028967906056, 1: 0.06209710320939444}, {0: 0.977507900874183, 1: 0.02249209912581691}, {0: 0.9801862148908839, 1: 0.01981378510911613}, {0: 0.9642020357560434, 1: 0.03579796424395662}, {0: 0.9663727015263743, 1: 0.033627298473625666}, {0: 0.8859702012473223, 1: 0.11402979875267771}, {0: 0.9586866934982315, 1: 0.04131330650176841}, {0: 0.9623146440112557, 1: 0.03768535598874437}, {0: 0.9298929992036218, 1: 0.07010700079637826}, {0: 0.9335352709009039, 1: 0.06646472909909606}, {0: 0.9021976306069932, 1: 0.09780236939300682}, {0: 0.9453965277787032, 1: 0.05460347222129675}, {1: 0.720226409263611, 0: 0.27977359073638897}]


In [7]:

test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 21 **


In [8]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train.csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev.csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
        )

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [9]:

def optimize_network(dropout,lr,neuronPct,neuronShrink,alpha,gamma):
    # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    output_weights_path = os.path.join(output_dir,  str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)+"_"+output_weights_name)
    print(f"** set output weights path to: {output_weights_path} **")
    checkpoint = ModelCheckpoint(

                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
    model = construct_network(dropout, neuronPct, neuronShrink)
    
    
    #model.compile(loss=focal_loss(gamma=gamma,alpha=alpha), optimizer=SGD(lr=lr))
    optimizer = SGD(lr=initial_learning_rate)
    model.compile(optimizer=optimizer,loss=focal_loss(gamma=gamma,alpha=alpha))

    #lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
    #lookahead.inject(model) # add into model
    callbacks = [
            checkpoint,
            keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
        ]
    
    model.summary()
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=class_weights,
            workers=generator_workers,
            shuffle=False,
        )
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    
    with open(test_log_path, "w") as f:
        for i in range(len(class_names)):
            try:
                score = roc_auc_score(y[:, i], y_hat[:, i])
                aurocs.append(score)
            except ValueError:
                score = 0
            f.write(f"{class_names[i]}: {score}\n")
        mean_auroc = float(np.mean(aurocs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}")
    

    print("iteration|auroc|alpha|dropout|gamma|learning_rate|neuronPct|neuronShrink")
    tensorflow.keras.backend.clear_session()
    time_took = time.time() - start_time
    return mean_auroc

        
    
    model.summary()
    


In [10]:
from new_bayes_opt.bayesian_optimization import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

# Bounded region of parameter space
pbounds = { 'gamma':(1.3, 3.0),
            'alpha':(0.5, 1.0),
            'dropout': (0.07323, 0.07324),
           'lr': (0.00923, 0.00924),
           'neuronPct': (0.1944 , 0.1945),
           'neuronShrink': (0.3521, 0.3522)
          }

#print(bounds.values())


In [11]:
optimizer = BayesianOptimization(
    f=optimize_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)

Kunci ['alpha', 'dropout', 'gamma', 'lr', 'neuronPct', 'neuronShrink']
Bound  [[0.5     1.     ]
 [0.07323 0.07324]
 [1.3     3.     ]
 [0.00923 0.00924]
 [0.1944  0.1945 ]
 [0.3521  0.3522 ]]


In [12]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
logger = JSONLogger(path="./without_noise12_1.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [13]:
def convert_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [14]:
start_time = time.time()
optimizer.maximize(init_points=5,acq="weightedei", n_iter=2, omega=0.9)
time_took = time.time() - start_time

Isine x [0.708511   0.0732372  1.30019444 0.00923302 0.19441468 0.35210923]
Konten (0.7085110023512871, 0.07323720324493442, 1.3001944371894862, 0.009233023325726319, 0.1944146755890817, 0.3521092338594769)
Masuk
** set output weights path to: ./experiments/0.07323720324493442_0.009233023325726319__0.1944146755890817_0.3521092338594769_weights.h5 **
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 342)               525654    
_________________________________________________________________
activation (Activation)      (None, 342)               0         
_________________________________________________________________
dropout (Dropout)            (None, 342)               0         
__________________________________

Epoch 6/7
75/76 [============================>.] - ETA: 0s - loss: 257.8157
Epoch 00006: val_loss improved from 286.83731 to 286.82307, saving model to ./experiments/0.07323345560727043_0.009235388167340033__0.19444191945144032_0.35216852195003967_weights.h5
76/76 [==============================] - 22s 296ms/step - loss: 257.5235 - val_loss: 286.8231
Epoch 7/7
75/76 [============================>.] - ETA: 0s - loss: 257.8077
Epoch 00007: val_loss improved from 286.82307 to 286.81917, saving model to ./experiments/0.07323345560727043_0.009235388167340033__0.19444191945144032_0.35216852195003967_weights.h5
21/21 [==============================] - 5s 260ms/step
** write log to ./experiments/0.07323345560727043_0.009235388167340033__0.19444191945144032_0.35216852195003967_test.log **
mean auroc: 0.8586829734932444
iteration|auroc|alpha|dropout|gamma|learning_rate|neuronPct|neuronShrink
Konten (0.5931301056888354, 0.07323345560727043, 1.974504706192139, 0.009235388167340033, 0.1944419194514

Epoch 4/7
75/76 [============================>.] - ETA: 0s - loss: 164.4850
Epoch 00004: val_loss improved from 184.21889 to 183.26927, saving model to ./experiments/0.07323198101489085_0.009239682615757194__0.19443134241781593_0.35216923226156693_weights.h5
76/76 [==============================] - 22s 294ms/step - loss: 164.2949 - val_loss: 183.2693
Epoch 5/7
75/76 [============================>.] - ETA: 0s - loss: 164.4062
Epoch 00005: val_loss improved from 183.26927 to 183.26262, saving model to ./experiments/0.07323198101489085_0.009239682615757194__0.19443134241781593_0.35216923226156693_weights.h5
76/76 [==============================] - 22s 294ms/step - loss: 164.2247 - val_loss: 183.2626
Epoch 6/7
75/76 [============================>.] - ETA: 0s - loss: 164.3877
Epoch 00006: val_loss improved from 183.26262 to 183.23833, saving model to ./experiments/0.07323198101489085_0.009239682615757194__0.19443134241781593_0.35216923226156693_weights.h5
76/76 [============================

Epoch 2/7
75/76 [============================>.] - ETA: 0s - loss: 198.9957
Epoch 00002: val_loss improved from 225.78198 to 225.68469, saving model to ./experiments/0.07323864575471516_0.00923__0.1944198563064689_0.3521895588218734_weights.h5
76/76 [==============================] - 22s 292ms/step - loss: 198.7904 - val_loss: 225.6847
Epoch 3/7
75/76 [============================>.] - ETA: 0s - loss: 196.7040
Epoch 00003: val_loss did not improve from 225.68469

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
76/76 [==============================] - 22s 292ms/step - loss: 196.5144 - val_loss: 225.6992
Epoch 4/7
75/76 [============================>.] - ETA: 0s - loss: 194.9411
Epoch 00004: val_loss improved from 225.68469 to 223.33409, saving model to ./experiments/0.07323864575471516_0.00923__0.1944198563064689_0.3521895588218734_weights.h5
76/76 [==============================] - 22s 292ms/step - loss: 194.7350 - val_loss: 223.3341
Epoch 5/7
75/76 [==

In [15]:
print(f"Total runtime: {convert_string(time_took)}")
print(optimizer.max)

Total runtime: 0:19:49.26
{'target': 0.8602087028848369, 'params': {'alpha': 0.9381945761480192, 'dropout': 0.07323894606663504, 'gamma': 1.4445751593286225, 'lr': 0.00923039054783233, 'neuronPct': 0.19441698304195645, 'neuronShrink': 0.35218781425034296}}


In [16]:
import gc
gc.collect()

4869

In [17]:
new_params = {} 
for t in optimizer.max.keys():
    print(t)
    new_params = optimizer.max.get(t)    

target
params


In [18]:
a = new_params.get('alpha')
d = new_params.get('dropout')
g = new_params.get('gamma')
l = new_params.get('lr')
np = new_params.get('neuronPct')
ns = new_params.get('neuronShrink')
new_params.update({'alpha':(float(a-0.025),float(a+0.025)),
                   'dropout':(float(d-0.025),float(d+0.025)),
                   'gamma':(float(g-0.025),float(g+0.025)),
                   'lr':(float(l),float(l+0.025)),                   
                   'neuronPct':(float(np-0.025),float(np+0.025)),
                   'neuronShrink':(float(ns-0.025),float(ns+0.025))
                  }
                 )

In [19]:
new_params

{'alpha': (0.9131945761480191, 0.9631945761480192),
 'dropout': (0.04823894606663504, 0.09823894606663505),
 'gamma': (1.4195751593286225, 1.4695751593286224),
 'lr': (0.00923039054783233, 0.034230390547832334),
 'neuronPct': (0.16941698304195646, 0.21941698304195645),
 'neuronShrink': (0.32718781425034293, 0.377187814250343)}

In [20]:
import csv
w = csv.writer(open("new_params.csv", "w"))
for key, val in new_params.items():
    w.writerow([key, val])